In [1]:
#!/usr/bin/env python

In [2]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [3]:
import wuml
import numpy as np
import torch
import torch
import torch.nn as nn

In [4]:
def costFunction(x, x̂, ẙ, y, ŷ, ind):	
#	x -> encoder -> x̂
#	x̂ -> encoder_linear_output -> ẙ	
#	x̂ -> decoder -> ŷ	
#	possible autoencoder objective λ could be 0
#	loss = (x - ŷ)ᒾ + λ * objective(ẙ, y)
#
#	This function can return 1 value or 3 values in a list
#	if return 1 value, just the loss
#	if return 3 values, [total_loss, reconstruction_loss, extra network from ẙ loss]
#
#	In this example, we perform both reconstruction and CE loss
#
	CE_loss = nn.CrossEntropyLoss() #weird pytorch, dim of y is 1, and ŷ is 20x3
	R = torch.sum((x - ŷ) ** 2)/(128*13)	#scaled by batch size times data dimension
	CE = CE_loss(ẙ, y)
	loss = R + CE
	return [loss, R, CE]

In [5]:
def costFunction2(x, x̂, ẙ, y, ŷ, ind):	
#	Example of just return 1 value, this is just a regular autoencoder
	return torch.sum((x - ŷ) ** 2)

In [6]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', 
					preprocess_data='center and scale', batch_size=128, label_type='discrete')

In [7]:
AE = wuml.autoencoder(12, data, default_depth=2, costFunction=costFunction, # costFunction and costFunction2 both works
						max_epoch=2000, encoder_output_weight_structure=[(3,'none')] ) 
AE.fit()
wuml.save_torch_network(AE, './autoencoder.pk')

	epoch: 2000, Total Loss/dimension: 0.0122, Reconstruct loss: 0.0120, Objective loss: 0.0002, Learning Rate: 0.00012500

Notice that the reconstruction did a reasonable job

In [8]:
ŷ = AE(data)
wuml.jupyter_print(ŷ[0:5,0:10])
wuml.jupyter_print(data.X[0:5,0:10])

[[ 1.3351 -0.4773  0.1916 -1.2497  1.84    0.8116  1.3817 -0.7238  1.223   0.1912]
 [ 0.2669 -0.531  -0.7638 -2.5457 -0.0008  0.604   0.6597 -0.8665 -0.4839 -0.3152]
 [ 0.4477  0.0875  0.9136 -0.1047  0.2687  0.7826  1.495  -0.3168  1.875   0.3804]
 [ 1.5818 -0.3229  0.4955 -0.8551  0.8829  2.4934  1.5586 -1.0254  1.06    1.1503]
 [ 0.2012  0.2561  1.8079  0.4646  1.2516  0.7823  0.7335  0.2286  0.3999 -0.3337]]


,0,1,2,3,4,5,6,7,8,9
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276


This is the bottleneck output which is d=12

In [9]:
x̂ = AE.reduce_dimension(data, output_type='wData')
wuml.jupyter_print(x̂.shape)

This is the objective network output 

In [10]:
ẙ = AE.objective_network(data)

Here we use the bottleneck to perform SVM classification

In [11]:
cf = wuml.classification(x̂, classifier='SVM')
wuml.jupyter_print(cf.result_summary(print_out=False))

,classifier,Train,Test
,SVM,0.994,1.0


Here we use the objective network output to perform LogisticRegression classification

In [12]:
cf = wuml.classification(ẙ, classifier='LogisticRegression')
wuml.jupyter_print(cf.result_summary(print_out=False))

,classifier,Train,Test
,LogisticRegression,1.0,1.0
